In [7]:
import os
import pandas as pd
import numpy as np
import re
import numpy as np
from scipy.stats import sem, t
from sklearn.utils import resample
from sklearn.metrics import roc_auc_score, average_precision_score
import seaborn as sns
import matplotlib.pyplot as plt

In [83]:
BRCA1_PROTIEN_STOP=1863
PALB2_PROTIEN_STOP=1183
BRCA2_PROTIEN_STOP=3418
RAD51C_PROTIEN_STOP=376
TP53_PROTIEN_STOP=393 
CHEK2_PROTIEN_STOP=543 
PTEN_PROTIEN_STOP=403 
BRCA1_transcript = "ENST00000357654"
BRCA2_transcript = "ENST00000380152"
PALB2_transcript = "ENST00000261584"
RAD51C_transcript = "ENST00000337432"
TP53_transcript = 'ENST00000269305'
CHEK2_transcript = 'ENST00000404276'
PTEN_transcript = 'ENST00000371953'



In [13]:
#dbNSFP
BRCA1_dbnsfp_data = "/research/bsi/projects/PI/tertiary/Couch_Fergus_coucf/s123456.general_utility/processing/thesis/dbNSFP/2023_11/BRCA1.dbNSFP4.7a_variant.chr17.20240413.tsv"
BRCA2_dbnsfp_data = "/research/bsi/projects/PI/tertiary/Couch_Fergus_coucf/s123456.general_utility/processing/thesis/dbNSFP/2023_11/BRCA2.dbNSFP4.7a_variant.chr13.20240314.tsv"
PALB2_dbnsfp_data = "/research/bsi/projects/PI/tertiary/Couch_Fergus_coucf/s123456.general_utility/processing/thesis/dbNSFP/2023_11/PALB2.dbNSFP4.7a_variant.chr16.20240314.tsv"
RAD51C_dbnsfp_data = "/research/bsi/projects/PI/tertiary/Couch_Fergus_coucf/s123456.general_utility/processing/thesis/dbNSFP/2023_11/RAD51C.dbNSFP4.7a_variant.chr17.20240413.tsv"
TP53_dbnsfp_data = "/research/bsi/projects/PI/tertiary/Couch_Fergus_coucf/s123456.general_utility/processing/thesis/dbNSFP/2023_11/TP53.dbNSFP4.7a_variant.chr17.20240414.tsv"
CHEK2_dbnsfp_data = "/research/bsi/projects/PI/tertiary/Couch_Fergus_coucf/s123456.general_utility/processing/thesis/dbNSFP/2023_11/CHEK2.dbNSFP4.7a_variant.chr22.20240414.tsv"
PTEN_dbnsfp_data = "/research/bsi/projects/PI/tertiary/Couch_Fergus_coucf/s123456.general_utility/processing/thesis/dbNSFP/2023_11/PTEN.dbNSFP4.7a_variant.chr17.20240414.tsv"

#disorder
BRCA1_disorder = '/research/bsi/projects/PI/tertiary/Couch_Fergus_coucf/s123456.general_utility/processing/thesis/disorder/mobidb_result_2024-06-09T20_44_18.054Z.BRCA1.tsv'
BRCA2_disorder = '/research/bsi/projects/PI/tertiary/Couch_Fergus_coucf/s123456.general_utility/processing/thesis/disorder/mobidb_result_2024-06-09T20_45_11.298Z.BRCA2.tsv'
PALB2_disorder = '/research/bsi/projects/PI/tertiary/Couch_Fergus_coucf/s123456.general_utility/processing/thesis/disorder/mobidb_result_2024-06-09T20_32_02.645Z.PALB2.tsv'
RAD51C_disorder = '/research/bsi/projects/PI/tertiary/Couch_Fergus_coucf/s123456.general_utility/processing/thesis/disorder/mobidb_result_2024-06-09T20_34_32.379Z.RAD51C.tsv'
TP53_disorder = '/research/bsi/projects/PI/tertiary/Couch_Fergus_coucf/s123456.general_utility/processing/thesis/disorder/mobidb_result_2024-09-11T21_03_59.549Z.TP53.tsv'
CHEK2_disorder = '/research/bsi/projects/PI/tertiary/Couch_Fergus_coucf/s123456.general_utility/processing/thesis/disorder/mobidb_result_2024-09-11T21_04_23.387Z.CHEK2.tsv'
PTEN_disorder = '/research/bsi/projects/PI/tertiary/Couch_Fergus_coucf/s123456.general_utility/processing/thesis/disorder/mobidb_result_2024-09-11T21_03_38.584Z.PTEN.tsv'

In [10]:
def three_to_one(aa_three):
    amino_acids = {
        'ALA': 'A', 'ARG': 'R', 'ASN': 'N', 'ASP': 'D',
        'CYS': 'C', 'GLU': 'E', 'GLN': 'Q', 'GLY': 'G',
        'HIS': 'H', 'ILE': 'I', 'LEU': 'L', 'LYS': 'K',
        'MET': 'M', 'PHE': 'F', 'PRO': 'P', 'SER': 'S',
        'THR': 'T', 'TRP': 'W', 'TYR': 'Y', 'VAL': 'V'
    }

    return amino_acids.get(aa_three.upper(), None)


def extract_integers(s):
    return [int(match.group()) for match in re.finditer(r'\d+', s)]

def preprocess_dbNSFP(dbNSFP_data, gene, transcript):
    map_class_dict = {"unknown" : 'VUS',
                      "Conflicting_interpretations_of_pathogenicity":'VUS',
                      "not_provided" : "VUS",
                      "not_provided": "VUS",
                      "Uncertain_significance" : "VUS",
                      "Pathogenic": "Abnormal",
                      "Likely_benign": "Normal",
                      "Benign": "Normal",
                      "Likely_pathogenic": "Abnormal",
                      "Pathogenic/Likely_pathogenic": "Abnormal",
                      "Benign/Likely_benign" : "Normal"}
    dbNSFP_df = pd.read_csv(dbNSFP_data, sep='\t')
    required_columns = ['aaref', 'aaalt', 'aapos', 'genename', 'Ensembl_transcriptid',
                        'clinvar_clnsig' ]

    is_rankscore_column = [column for column in dbNSFP_df.columns if column.endswith('rankscore')]
    values_to_remove = ['LINSIGHT_rankscore', 'GenoCanyon_rankscore', 'LIST-S2_rankscore', 'fathmm-MKL_coding_rankscore',
                   'fathmm-XF_coding_rankscore']
    is_rankscore_column = [item for item in is_rankscore_column if item not in values_to_remove]
    required_columns += is_rankscore_column

    dbNSFP_req_df = dbNSFP_df[required_columns]
    dbNSFP_req_df['aapos'] = dbNSFP_req_df['aapos'].str.split(';')
    dbNSFP_req_df['Ensembl_transcriptid'] = dbNSFP_req_df['Ensembl_transcriptid'].str.split(';')
    dbNSFP_req_df['genename'] = dbNSFP_req_df['genename'].str.split(';')
    columns_to_explode = [ 'aapos','Ensembl_transcriptid', 'genename']
    dbNSFP_req_df = dbNSFP_req_df.apply(lambda x: x.explode() if x.name in columns_to_explode else x)
    dbNSFP_req_df = dbNSFP_req_df[(dbNSFP_req_df['genename'] == gene) &
                                  (dbNSFP_req_df['Ensembl_transcriptid'] == transcript)]

    dbNSFP_req_df['aapos'] = dbNSFP_req_df['aapos'].astype(int)
    dbNSFP_req_df = dbNSFP_req_df[dbNSFP_req_df['aapos'] >=0]
    dbNSFP_req_df['aapos'] = dbNSFP_req_df['aapos'].astype(str)
    dbNSFP_req_df['mutations'] = dbNSFP_req_df['aaref'] + dbNSFP_req_df['aapos'] + dbNSFP_req_df['aaalt']
    
    dbNSFP_req_df = dbNSFP_req_df.replace(".", np.nan)
    dbNSFP_req_df['aapos'] = dbNSFP_req_df['aapos'].astype(int)
    columns_to_fillna = is_rankscore_column
    dbNSFP_req_df[columns_to_fillna] = dbNSFP_req_df[columns_to_fillna].fillna(0)

    dbNSFP_req_df['clinvar_clnsig'] = dbNSFP_req_df['clinvar_clnsig'].fillna('unknown')
    dbNSFP_req_df['ClinVar_Class'] = dbNSFP_req_df['clinvar_clnsig'].map(map_class_dict)
    dbNSFP_req_df = dbNSFP_req_df.drop_duplicates().reset_index(drop=True)
    #bNSFP_req_df = dbNSFP_req_df[['mutations', 'ClinVar_Class']]
    dbNSFP_req_clinvar_df = dbNSFP_req_df[(dbNSFP_req_df['ClinVar_Class'] == "Abnormal") | (dbNSFP_req_df['ClinVar_Class'] == "Normal")]
    dbNSFP_req_clinvar_df = dbNSFP_req_clinvar_df.drop_duplicates()
    dbNSFP_req_clinvar_df = dbNSFP_req_clinvar_df.reset_index(drop=True)
    return dbNSFP_req_df, dbNSFP_req_clinvar_df

In [20]:
BRCA1_all_df, BRCA1_dbNSFP_df = preprocess_dbNSFP(BRCA1_dbnsfp_data, 'BRCA1', BRCA1_transcript)
BRCA2_all_df, BRCA2_dbNSFP_df = preprocess_dbNSFP(BRCA2_dbnsfp_data, 'BRCA2', BRCA2_transcript)
PALB2_all_df, PALB2_dbNSFP_df = preprocess_dbNSFP(PALB2_dbnsfp_data, 'PALB2', PALB2_transcript)
RAD51C_all_df, RAD51C_dbNSFP_df = preprocess_dbNSFP(RAD51C_dbnsfp_data, 'RAD51C', RAD51C_transcript)
TP53_all_df, TP53_dbNSFP_df = preprocess_dbNSFP(TP53_dbnsfp_data, 'TP53', TP53_transcript)
CHEK2_all_df, CHEK2_dbNSFP_df = preprocess_dbNSFP(CHEK2_dbnsfp_data, 'CHEK2', CHEK2_transcript)
PTEN_all_df, PTEN_dbNSFP_df = preprocess_dbNSFP(PTEN_dbnsfp_data, 'PTEN', PTEN_transcript)

/research/bsi/projects/PI/tertiary/Couch_Fergus_coucf/s123456.general_utility/python_virtual/lib/python3.7/site-packages/ipykernel_launcher.py:1: DtypeWarning: Columns (49,50,52,68,158,159,161,162,163,164,165,166,188,189,194,195,199,200) have mixed types.Specify dtype option on import or set low_memory=False.
  """Entry point for launching an IPython kernel.
/research/bsi/projects/PI/tertiary/Couch_Fergus_coucf/s123456.general_utility/python_virtual/lib/python3.7/site-packages/ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/research/bsi/projects/PI/tertiary/Couch_Fergus_coucf/s123456.general_utility/python_virtual/lib/python3.7/site-packages/ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a

In [69]:
def process_disorder_data(dbNSFP_df, gene_name, stop, disorder_ranges=None):
    disorder_df = pd.DataFrame({'aapos': range(1, stop + 1)})
    disorder_df['disorder'] = 0
    if disorder_ranges:
        for start, end in disorder_ranges:
            disorder_df.loc[(disorder_df['aapos'] >= start) & (disorder_df['aapos'] <= end), 'disorder'] = 1
    dbNSFP_df['aapos'] = dbNSFP_df['aapos'].astype(int)
    merged_df = pd.merge(disorder_df, dbNSFP_df, on='aapos', how='right')
    merged_df = merged_df[(merged_df['disorder'] == 1) & (merged_df['aaalt'] != 'X')]
    merged_df['GENE'] = gene_name
    print(f"Processed {gene_name} gene.")
    
    return merged_df

In [84]:
# Example usage:
BRCA1_disorder_ranges = [(100, 1649)]
BRCA2_disorder_ranges = [(40, 284), (358, 381), (2430, 2450), (3393, 3418)]
PALB2_disorder_ranges = [(835, 853), (879, 882), (950, 955), (996, 997), (1054, 1056), (1076, 1087), (1155, 1156)]
RAD51C_disorder_ranges = [(1, 12), (71, 88), (351, 376)]
TP53_disorder_ranges = [(1, 97),(288,312),(361,393)]
PTEN_disorder_ranges = [(286,309), (353,403)]
CHEK2_disorder_ranges = [(1,69),(89,90),(504,513),(532,543)]


BRCA1_dbNSFP_disorder_df = process_disorder_data( BRCA1_dbNSFP_df, "BRCA1", BRCA1_PROTIEN_STOP, disorder_ranges=BRCA1_disorder_ranges)
BRCA2_dbNSFP_disorder_df = process_disorder_data(BRCA2_dbNSFP_df, "BRCA2", BRCA2_PROTIEN_STOP, disorder_ranges=BRCA2_disorder_ranges)
PALB2_dbNSFP_disorder_df = process_disorder_data(PALB2_dbNSFP_df, "PALB2", PALB2_PROTIEN_STOP, disorder_ranges=PALB2_disorder_ranges)
RAD51C_dbNSFP_disorder_df = process_disorder_data(RAD51C_dbNSFP_df, "RAD51C", RAD51C_PROTIEN_STOP, disorder_ranges=RAD51C_disorder_ranges)
TP53_dbNSFP_disorder_df = process_disorder_data(TP53_dbNSFP_df, "TP53", TP53_PROTIEN_STOP, disorder_ranges=TP53_disorder_ranges)
CHEK2_dbNSFP_disorder_df = process_disorder_data(CHEK2_dbNSFP_df, "CHEK2",CHEK2_PROTIEN_STOP,disorder_ranges=CHEK2_disorder_ranges)
PTEN_dbNSFP_disorder_df = process_disorder_data(PTEN_dbNSFP_df, "PTEN", PTEN_PROTIEN_STOP, disorder_ranges=PTEN_disorder_ranges)


Processed BRCA1 gene.
Processed BRCA2 gene.
Processed PALB2 gene.
Processed RAD51C gene.
Processed TP53 gene.
Processed CHEK2 gene.
Processed PTEN gene.


In [85]:
TP53_dbNSFP_df['aapos'].max()

393

In [86]:
disorder_dbNSFP_df = pd.concat([BRCA1_dbNSFP_disorder_df, BRCA2_dbNSFP_disorder_df, 
                               PALB2_dbNSFP_disorder_df, RAD51C_dbNSFP_disorder_df,
                               TP53_dbNSFP_disorder_df, CHEK2_dbNSFP_disorder_df,
                               PTEN_dbNSFP_disorder_df], axis=0)

In [87]:
disorder_dbNSFP_df[['ClinVar_Class', 'GENE']].value_counts()

ClinVar_Class  GENE 
Normal         BRCA1    266
               TP53      68
               BRCA2     28
Abnormal       BRCA1     11
               BRCA2      5
Normal         PTEN       4
               CHEK2      3
Abnormal       TP53       2
               CHEK2      1
Normal         PALB2      1
dtype: int64